# stream-bitflip
En este desafío, el servidor permite a un usuario registrarse con un correo electrónico y un conjunto de datos adicionales, y devuelve un perfil cifrado con un algoritmo de cifrado de flujo.

Si el usuario registró el correo usuario@example.com y puso como datos “Juan P. Usuario”, el perfil tendrá la forma:

user=usuario@example.com;data=Juan P. Usuario;role=user

Es decir, está compuesto por un conjunto de pares atributo-valor, de la forma <atributo>=<valor>, y separados por punto y coma.

El servidor devuelve el perfil cifrado, precedido por un nonce de 16 bytes, y codificado en base64.

El desafío es alterar el mensaje cifrado de manera tal que al descifrarlo incluya el par role=admin. Esta alteración es posible porque los cifradores de flujo son maleables. Es posible modificar el texto cifrado de manera tal que al descifrarlo el texto claro haya sido modificado de manera predecible.

Este tipo de ataque se denomina bit flipping, porque el cambio de un bit en el texto cifrado provoca un cambio del mismo bit en el texto claro.

Para obtener los textos cifrados, se debe hacer un requerimiento POST a una URL de la forma:

https://ciberseguridad.diplomatura.unc.edu.ar/cripto/stream-bitflip/<email>/register
        

dónde <email> debe ser reemplazado por una dirección de correo electrónico registrada.

El contenido debe ser de tipo FORM, con dos campos:

- Un campo email, con una dirección de correo a registrar. No es necesario que sea la misma dirección utilizada para acceder al desafío.
- Un campo data, con datos arbitrarios, codificado en base64. Los datos no pueden contener el carácter ‘;’ ni el carácter ‘=’.

La respuesta será un perfil como el descripto, cifrado, concatenado con el nonce y codificado en base64.

Si en lugar de hacer un requerimiento POST se hace un requerimiento GET, muestra un formulario que permite cargar los campos requeridos.

In [49]:
import requests
import base64
SERVER = "https://ciberseguridad.diplomatura.unc.edu.ar/cripto"
EMAIL = "alvarmaciel@gmail.com"
form_data = {
    "email": "alvarmaciel@gmail.com",
    "data": base64.b64encode(b"XXXXXXXXXX").decode('utf-8')
}
print(form_data["data"])
def get_challenge(challenge:str) -> str:
    response =  requests.post(f"{SERVER}/{challenge}/{EMAIL}/register", files=form_data)
    return response

WFhYWFhYWFhYWA==


In [50]:
response = get_challenge("stream-bitflip")
print(response.status_code)
print(response.content)
print(response.request.body)


200
b'fYFJCXcOKB341P10px6xCymF53tQQPUydLHC5jkb7D0ACxfRZEJWwayJd12cMpAh7SEiS2FNMqAGWXenGPDYz8Ai/Gw='
b'--c4e55a089941c16b4ae526cef8053130\r\nContent-Disposition: form-data; name="email"; filename="email"\r\n\r\nalvarmaciel@gmail.com\r\n--c4e55a089941c16b4ae526cef8053130\r\nContent-Disposition: form-data; name="data"; filename="data"\r\n\r\nWFhYWFhYWFhYWA==\r\n--c4e55a089941c16b4ae526cef8053130--\r\n'


In [51]:
encrypted = base64.b64decode(response.content)
nonce = encrypted[:16]
ciphertext = encrypted[16:]

print(f"Nonce length: {len(nonce)}")
print(f"Ciphertext length: {len(ciphertext)}")
print(f"Ciphertext (hex): {ciphertext.hex()}")

Nonce length: 16
Ciphertext length: 52
Ciphertext (hex): 2985e77b5040f53274b1c2e6391bec3d000b17d1644256c1ac89775d9c329021ed21224b614d32a0065977a718f0d8cfc022fc6c


In [52]:
plain_text = "user=alvarmaciel@gmail.com;data=XXXXXXXXXX;role=user"

In [53]:
print(f"Plain text lenghy: {len(plain_text)}")

Plain text lenghy: 52


In [55]:
plain_text[-4:]

'user'

In [60]:
plain_text[-11:]

'X;role=user'

In [62]:
original = b"X;role=user"  
target   = b";role=admin" 

In [63]:
delta = bytes([o ^ t for o, t in zip(original, target)])
print(f"Delta (hex): {delta.hex()}")

Delta (hex): 63491d0309585c111e0c1c


In [64]:
modified_ciphertext = bytearray(ciphertext)
start_pos = len(ciphertext) - 11

for i in range(11):
    modified_ciphertext[start_pos + i] ^= delta[i]

modified_ciphertext = bytes(modified_ciphertext)

In [68]:
modified_ciphertext

b')\x85\xe7{P@\xf52t\xb1\xc2\xe69\x1b\xec=\x00\x0b\x17\xd1dBV\xc1\xac\x89w]\x9c2\x90!\xed!"KaM2\xa0\x06:>\xba\x1b\xf9\x80\x93\xd1<\xf0p'

In [70]:
modified_encrypted = nonce + modified_ciphertext
modified_b64 = base64.b64encode(modified_encrypted).decode('utf-8')

print(f"Modified (base64): {modified_b64}")

Modified (base64): fYFJCXcOKB341P10px6xCymF53tQQPUydLHC5jkb7D0ACxfRZEJWwayJd12cMpAh7SEiS2FNMqAGOj66G/mAk9E88HA=


In [71]:
challenge_response = requests.post(
    f"{SERVER}/stream-bitflip/{EMAIL}/answer",
    files = { "message":modified_b64}
)
print(f"Status code: {challenge_response.status_code}")
print(challenge_response.text)

Status code: 200
¡Ganaste!

